# Pattern Driven Stock Analysis

Finding Similar Stocks using Graph Analytis in Neo4j.


# Steps

ETL Script is at Stock Analysis ETL.ipynb

1. [X] Import Stock Data into Neo4j
2. [X] Create [:NEXT] relationship property between nodes
   
4. [X] Find similarity using trade volume among nodes: Create [:VOL_SIMILAR]  relationship among STOCK Nodes
5. [X] Community detection based on Louvain algorithm using [:VOL_SIMILAR] relationship: Create community property

![Next.png](Next.png)

### Create NEXT_DAY relationship among StockTradingDay nodes

    MATCH (s:Stock)-[r:TRADING_DAY]->(td:StockTradingDay) 
    WITH s, td
    ORDER BY td.Date ASC
    WITH s, collect(td) as nodes
    CALL graph_util.chain_nodes(nodes, "NEXT_DAY")
    YIELD connections
    RETURN nodes, connections;

###   Creating Date Nodes

    MATCH (td:StockTradingDay)  
    WITH DISTINCT td.Date as date
    ORDER BY date ASC WITH date CREATE (d:Date {id:date});


### Relating Date Nodes with Stock Trading day

    MATCH (td:StockTradingDay)  
    MATCH (d:Date) where td.Date = d.id
    MERGE (d)-[:DATE]->(td);

 ### Creating DATE_NEXT relationship among Date nodes
    
    MATCH (d:Date)
    WITH d
    ORDER BY d.id ASC
    WITH collect(d) as nodes
    CALL graph_util.chain_nodes(nodes, "DATE_NEXT")
    YIELD connections
    RETURN nodes, connections;

    MATCH (td1:StockTradingDay)-[r:NEXT_DAY]->(td2:StockTradingDay) 
    SET r.volumeGap = (td2.Volume-td1.Volume);

### Obtain stock volumes of all the dates 

Down load the result as CSV and rename it to 'StockDateVolume.csv'


    MATCH (d:Date) 
    MATCH (s:Stock) 
    CALL {
        WITH s,d
        OPTIONAL MATCH (s)-[:TRADING_DAY]->(td:StockTradingDay)<-[:DATE]-(d)
        RETURN
        CASE 
            WHEN td.Volume is null THEN toFloat(0)
            ELSE toFloat(td.Volume)
        END as volume
        }
    RETURN s.id, d.id, volume;

# Stock Similarity using Dynamic time warping (DTW)

In [1]:
STOCK_DATA="NASDAQ_DATA"
import pandas as pd

DATA_FILE_NAME = 'StockDateVolume.csv'
StockVol_DF = pd.read_csv('{path}/{file}'.format(path=STOCK_DATA,file=DATA_FILE_NAME),delimiter=',')
StockVol_DF

,stockId,date,volume
0,MRNA,2010-01-04,0
1,TCOM,2010-01-04,3325200
2,ADI,2010-01-04,2102700
3,ADP,2010-01-04,3930120
4,ADSK,2010-01-04,2228600
...,...,...,...
300181,PAYX,2021-09-10,1257600
300182,PCAR,2021-09-10,2495500
300183,PDD,2021-09-10,4800600
300184,PYPL,2021-09-10,3811700


In [9]:
stocks = StockVol_DF.stockId.unique()
stocks

array(['MRNA', 'TCOM', 'ADI', 'ADP', 'ADSK', 'AMD', 'AMGN', 'ANSS',
       'ASML', 'BIIB', 'CDNS', 'CERN', 'CHTR', 'CMCSA', 'CPRT', 'CRWD',
       'CSX', 'CTAS', 'CTSH', 'DOCU', 'DXCM', 'EA', 'FAST', 'FB', 'FOXA',
       'GOOG', 'GOOGL', 'INTC', 'JD', 'LRCX', 'LULU', 'MCHP', 'MDLZ',
       'MELI', 'MRVL', 'MTCH', 'MU', 'REGN', 'SBUX', 'SNPS', 'SPLK',
       'TEAM', 'TXN', 'VRSK', 'WBA', 'XEL', 'ZM', 'NFLX', 'NTES', 'NVDA',
       'NXPI', 'ORLY', 'PEP', 'PTON', 'ROST', 'SGEN', 'SIRI', 'SWKS',
       'TMUS', 'TSLA', 'VRSN', 'VRTX', 'WDAY', 'XLNX', 'AAPL', 'ADBE',
       'AEP', 'ALGN', 'AMAT', 'AMZN', 'ATVI', 'AVGO', 'BIDU', 'BKNG',
       'CDW', 'CHKP', 'COST', 'CSCO', 'DLTR', 'EBAY', 'EXC', 'FISV',
       'FOX', 'GILD', 'HON', 'IDXX', 'ILMN', 'INCY', 'INTU', 'ISRG',
       'KDP', 'KHC', 'KLAC', 'MAR', 'MNST', 'MSFT', 'OKTA', 'PAYX',
       'PCAR', 'PDD', 'PYPL', 'QCOM'], dtype=object)

In [58]:
import csv
import itertools
from tslearn.metrics import dtw as ts_dtw
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
sc = StandardScaler()
enc = OrdinalEncoder()

from ipywidgets import IntProgress
from IPython.display import display
import time




def get_stock_ts(stockid,df):
    DF = df[df['stockId'] == stockid]
    DV = DF.values
    enc.fit(DV)
    data = enc.transform(DV)
    data_std = sc.fit_transform(data)
    return data_std


c = []
c.extend(itertools.combinations(stocks, 2))
filename = "stock_similarity.csv"

# display progress bar
p = IntProgress(min=0, max=len(c),description='Processing:',) 
display(p)

with open(filename, 'w') as csvfile: 
    fields = ['stock1', 'stock2', 'similarity'] 
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(fields) 
    
    for i in range(0, len(c)):
        s1 = c[i][0]
        s2 = c[i][1]
        data_std = get_stock_ts(s1,StockVol_DF)
        data_std2 = get_stock_ts(s2,StockVol_DF)
        distance = ts_dtw(data_std, data_std2)
    
        row = [s1,s2,(1/distance*10)]

        # writing the data rows 
        csvwriter.writerow(row)
        p.value += 1



IntProgress(value=0, description='Processing:', max=5151)

In [59]:
stock_similarity = pd.read_csv(filename,delimiter=',')
# stock_similarity['similarity'] =  1/stock_similarity['similarity']*10
stock_similarity.head()

,stock1,stock2,similarity
0,MRNA,TCOM,0.171130
1,MRNA,ADI,0.176612
2,MRNA,ADP,0.159324
3,MRNA,ADSK,0.160290
4,MRNA,AMD,0.241302


# Load the similarity data to Memgraph

    LOAD CSV FROM 'http://fileserver:4858/stock_similarity.csv' WITH HEADER AS row
    MATCH (s1:Stock {id:row.stock1})
    MATCH (s2:Stock {id:row.stock2})
    MERGE (s1)-[rv:VOL_SIMILAR {weight:toFloat(row.similarity)}]->(s2);


### Community detection
Using Louvain algorithm on [:VOL_SIMILAR] relationship to Create VolumeCommunity property

https://memgraph.com/docs/advanced-algorithms/available-algorithms/community_detection

https://memgraph.com/blog/identify-patterns-and-anomalies-with-community-detection-graph-algorithm

    MATCH  p=((s1:Stock)-[e:VOL_SIMILAR]->(s2:Stock))
    WITH project(p) AS subgraph 
    CALL community_detection.get(subgraph)
    YIELD node, community_id
    SET node.community = community_id
    RETURN node, community_id
    ORDER BY node.id;


In [57]:
# Networkx Community Detection

import networkx as nx
import pandas as pd

stock_similarity = stock_similarity.rename(columns={'stock1': 'source', 'stock2': 'target','similarity': 'weight'})

Graphtype = nx.Graph()
G = nx.from_pandas_edgelist(stock_similarity, edge_attr='weight', create_using=Graphtype)
# community = nx.community.louvain_partitions(G,weight="weight",resolution=0)
nx.community.louvain_communities(G, seed=123)

[{'AAPL',
  'AMD',
  'CDW',
  'CHTR',
  'CRWD',
  'DOCU',
  'DXCM',
  'FB',
  'FOX',
  'FOXA',
  'JD',
  'KHC',
  'MRNA',
  'NXPI',
  'OKTA',
  'PDD',
  'PTON',
  'PYPL',
  'SPLK',
  'TEAM',
  'TSLA',
  'WDAY',
  'ZM'},
 {'ADBE',
  'ADP',
  'ADSK',
  'AMAT',
  'AMGN',
  'AMZN',
  'ASML',
  'BIDU',
  'BIIB',
  'BKNG',
  'CDNS',
  'CHKP',
  'CMCSA',
  'COST',
  'CSCO',
  'CSX',
  'CTAS',
  'CTSH',
  'EA',
  'EBAY',
  'FISV',
  'GILD',
  'GOOG',
  'GOOGL',
  'HON',
  'ILMN',
  'INTC',
  'INTU',
  'ISRG',
  'KLAC',
  'MAR',
  'MDLZ',
  'MNST',
  'MRVL',
  'MSFT',
  'MTCH',
  'NFLX',
  'NVDA',
  'ORLY',
  'PAYX',
  'PCAR',
  'PEP',
  'QCOM',
  'ROST',
  'SBUX',
  'SIRI',
  'SWKS',
  'TXN',
  'VRSN',
  'WBA',
  'XLNX'},
 {'ADI',
  'AEP',
  'ALGN',
  'ANSS',
  'ATVI',
  'AVGO',
  'CERN',
  'CPRT',
  'DLTR',
  'EXC',
  'FAST',
  'IDXX',
  'INCY',
  'KDP',
  'LRCX',
  'LULU',
  'MCHP',
  'MELI',
  'MU',
  'NTES',
  'REGN',
  'SGEN',
  'SNPS',
  'TCOM',
  'TMUS',
  'VRSK',
  'VRTX',
  'XEL'}]

![community.png](community.png)